In [ ]:
import os
import math

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import sklearn.gaussian_process as gp
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import commonmodules as cm

def compute_and_plot (df, vib_values , temp_values, testsetvib):

  cm.plotfull3dcurve (df, vib_values, temp_values)

  train_xy, train_z, test_xy, test_z = cm.get_train_and_test_rmv (temp_values, vib_values, \
    df, testsetvib)

  fig = plt.figure()
  ax = fig.add_subplot(projection='3d')

  for i in range(train_z.shape[0]):
    x = train_xy[i,0]
    y = train_xy[i,1]
    z = train_z[i]
    ax.scatter(x, y, z, marker="o", color="g")

  for i in range(test_z.shape[0]):
    x = test_xy[i,0]
    y = test_xy[i,1]
    z = test_z[i]
    ax.scatter(x, y, z, marker="o", color="r")

  ax.set_xlabel('X Label')
  ax.set_ylabel('Y Label')
  ax.set_zlabel('Z Label')
  plt.gcf().set_size_inches(20, 15)

  plt.show()

  model = cm.build_model_GP_2 (train_xy, train_z)

  z_pred, std = model.predict(train_xy, return_std=True)
  trainmse = 0.0
  cont = 0.0
  
  for i in range(train_z.shape[0]):
    x = train_xy[i,0]
    y = train_xy[i,1]
    z = train_z[i]
    zpred = z_pred[i]
    zstd = std[i]
    
    trainmse += (zpred-z)**2
    cont += 1.0

    #print("Train, %10.7f , %10.7f , %10.7f , %10.7f , %10.7f"%(z, y, z, zpred, zstd))

  trainmse = trainmse/cont

  print("X , Y , True Value , Predicted Value , STD")

  z_pred, std = model.predict(test_xy, return_std=True)
  mse = 0.0
  cont = 0.0
  for i in range(test_z.shape[0]):
    x = test_xy[i,0]
    y = test_xy[i,1]
    z = test_z[i]
    zpred = z_pred[i]
    zstd = std[i]
    
    mse += (zpred-z)**2
    cont += 1.0

    print("%10.7f , %10.7f , %10.7f , %10.7f , %10.7f"%(x, y, z, zpred, zstd))

  mse = mse/cont

  print("Removed random values  MSE ", mse, " , TrainMSE ,", trainmse, flush=True)

In [ ]:
filename = "dv1.xlsx"
df, vib_values , temp_values = cm.filterinitialset (filename)
testsetvib = [34, 36]

compute_and_plot (df, vib_values , temp_values, testsetvib)

In [14]:
filename = "dv2.xlsx"
df, vib_values , temp_values = cm.filterinitialset (filename)
testsetvib = [34, 36]

compute_and_plot (df, vib_values , temp_values, testsetvib)

KeyboardInterrupt: 

In [ ]:
filename = "dv3.xlsx"
df, vib_values , temp_values = cm.filterinitialset (filename)
testsetvib = [34, 36]

compute_and_plot (df, vib_values , temp_values, testsetvib)